# Artificial Neural Network

## Part1 - Importing the libraries

In [64]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

### Part 2 - Importing and preparing the dataset

### Dataset overview

#### Basic atrributes:
- Gender
- Age
- Height
- Weight

#### The attributes related with eating habits are
- FAVC - Frequent consumption of high caloric food  
- FCVC  - Frequency of consumption of vegetables 
- NCP - Number of main meals 
- CAEC - Consumption of food between meals 
- CH20 - Consumption of water daily 
- CALC Consumption of alcohol

#### The attributes related with the physical condition are:
- SCC - Calories consumption monitoring
- FAF - Physical activity frequency 
- TUE - Time using technology devices 
- MTRANS - Transportation used 

#### The label:
- NObesity - the target variable is made up of the independant variables Height and Weight. There are 7 unique values - 'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II', 'Obesity_Type_I', 'Insufficient_Weight', 'Obesity_Type_II', 'Obesity_Type_III'

#### Goal:
- Tries to predict level of obesity based on eating habits and physical condition

For more information on the data please see the EDA notebook.

In [65]:
dataset = pd.read_csv('./data/modified.csv');
X_wHW = dataset.iloc[:, :-1].values

In [66]:
dataset_no_H_W = pd.read_csv('./data/modified_no_H_W.csv');
X_nHW = dataset.iloc[:, :-1].values

### Encoding categorical data

The independable variables is encoded during the EDA

With OneHotEncoder for each unique value of an attribute a new column is created. Having 7 categories for our output means that for each y there are 7 values - 6 zeros and 1 one as shown below.

In [67]:
encoder = OneHotEncoder()
y_wHW = encoder.fit_transform(dataset[['NObesity']]).toarray()

In [68]:
print(y_wHW[0])

[0. 1. 0. 0. 0. 0. 0.]


In [70]:
encoder_nHW = OneHotEncoder()
y_nHW = encoder.fit_transform(dataset_no_H_W[['NObesity']]).toarray()

In [71]:
print(y_nHW[0])

[0. 1. 0. 0. 0. 0. 0.]


### Splitting the dataset into the Training set and Test set

In [72]:
X_train_wHW, X_test_wHW, y_train_wHW, y_test_wHW = train_test_split(X_wHW, y_wHW, test_size = 0.3, random_state = 0)

In [73]:
X_train_nHW, X_test_nHW, y_train_nHW, y_test_nHW = train_test_split(X_nHW, y_nHW, test_size = 0.3, random_state = 0)

There are several things about this step that are worth explaining. First and foremost, it is performed after the dataset is split into a training set and a test set. The reason for that is to simulate a real-world scenario with the test set.

The training set is fitted and transformed and the test set is just transformed. The reason for that is because I want them to be in the same format but to avoid the already mentioned 'information leakage'.

### Future Scaling

In [74]:
# Absolutely fundamental for NN. Should be performed it to all variables no matter of the encoding
sc_wHW = StandardScaler()
X_train_wHW = sc_wHW.fit_transform(X_train_wHW)
X_test_wHW = sc_wHW.transform(X_test_wHW)

In [75]:
sc_nHW = StandardScaler()
X_train_nHW = sc_nHW.fit_transform(X_train_nHW)
X_test_nHW = sc_nHW.transform(X_test_nHW)

## Part 3 - Building the function that will be used 

### Function that creates an ANN model

In [76]:
def create_model(units, optimizer):
    model = Sequential()
    # Input layer and the first hidden layer
    model.add(Dense(input_dim=16, units = units, activation='relu'))
    # Second hidden layer
    model.add(Dense(units =units, activation='relu'))
    # Output layer
    model.add(Dense(units = 7, activation='sigmoid'))
    # Compiling the model
    model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

### Fucntion for plotting the accuracy and the loss of the model

In [77]:
def plot_acc_loss(history_one, title_one, history_two, title_two):
    fig, ((ax1, ax2),( ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(30, 10))

    # summarize history for accuracy
    ax1.plot(history_one.history['accuracy'])
    ax1.plot(history_one.history['val_accuracy'])
    ax1.set_title(f'{title_one} accuracy')
    ax1.set_ylabel('accuracy')
    ax1.set_xlabel('epoch')
    ax1.legend(['train', 'test'], loc='upper left')

    # summarize history for accuracy
    ax2.plot(history_two.history['accuracy'])
    ax2.plot(history_two.history['val_accuracy'])
    ax2.set_title(f'{title_two} accuracy')
    ax2.set_ylabel('accuracy')
    ax2.set_xlabel('epoch')
    ax2.legend(['train', 'test'], loc='upper left')

    # summarize history for loss
    ax3.plot(history_one.history['loss'])
    ax3.plot(history_one.history['val_loss'])
    ax3.set_title(f'{title_one} loss')
    ax3.set_ylabel('loss')
    ax3.set_xlabel('epoch')
    ax3.legend(['train', 'test'], loc='upper left')

    # summarize history for loss
    ax4.plot(history_two.history['loss'])
    ax4.plot(history_two.history['val_loss'])
    ax4.set_title(f'{title_two} loss')
    ax4.set_ylabel('loss')
    ax4.set_xlabel('epoch')
    ax4.legend(['train', 'test'], loc='upper left')

    plt.show()

## Part 4 - Training the ANN model with Height and Weight

### Training the ANN on the training set and evaluate it on the test set

In [78]:
#32 is commonly used for batch size
#the epochs values should not be too small as I read
ann_wHW = create_model(units=10, optimizer='adam')
history_wHW = ann_wHW.fit(x = X_train_wHW, y=y_train_wHW, batch_size = 32, epochs = 100, validation_data=(X_test_wHW, y_test_wHW))

Epoch 1/100
45/45 [==============================] - 1s 7ms/step - loss: 1.9272 - accuracy: 0.2370 - val_loss: 1.8952 - val_accuracy: 0.2447
Epoch 2/100
45/45 [==============================] - 0s 3ms/step - loss: 1.7729 - accuracy: 0.3301 - val_loss: 1.7657 - val_accuracy: 0.2966
Epoch 3/100
45/45 [==============================] - 0s 5ms/step - loss: 1.6566 - accuracy: 0.3711 - val_loss: 1.6648 - val_accuracy: 0.3485
Epoch 4/100
45/45 [==============================] - 0s 3ms/step - loss: 1.5575 - accuracy: 0.4163 - val_loss: 1.5773 - val_accuracy: 0.3955
Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 1.4716 - accuracy: 0.4489 - val_loss: 1.4997 - val_accuracy: 0.4117
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 1.3932 - accuracy: 0.4830 - val_loss: 1.4255 - val_accuracy: 0.4522
Epoch 7/100
45/45 [==============================] - 0s 3ms/step - loss: 1.3204 - accuracy: 0.5240 - val_loss: 1.3576 - val_accuracy: 0.4797
Epoch 8/100
4

In [80]:
test_pred = ann_wHW.predict(X_test_wHW)
test_pred[0]

20/20 [==============================] - 0s 1ms/step


array([9.9687976e-01, 9.9998641e-01, 2.0907356e-12, 7.3958299e-21,
       9.0967368e-16, 9.4924045e-01, 2.6256372e-03], dtype=float32)

## Part 5 - Training the ANN model without Height and Weight

### Training the ANN on the training set and evaluate it on the test set

In [79]:
ann_nHW = create_model(units=10, optimizer='adam')
history_nHW = ann_nHW.fit(x = X_train_nHW, y=y_train_nHW, batch_size = 32, epochs = 100, validation_data=(X_test_nHW, y_test_nHW))

Epoch 1/100
45/45 [==============================] - 1s 8ms/step - loss: 1.9392 - accuracy: 0.2342 - val_loss: 1.8699 - val_accuracy: 0.2707
Epoch 2/100
45/45 [==============================] - 0s 4ms/step - loss: 1.7937 - accuracy: 0.2988 - val_loss: 1.7578 - val_accuracy: 0.2836
Epoch 3/100
45/45 [==============================] - 0s 4ms/step - loss: 1.6744 - accuracy: 0.3127 - val_loss: 1.6569 - val_accuracy: 0.3193
Epoch 4/100
45/45 [==============================] - 0s 5ms/step - loss: 1.5662 - accuracy: 0.3537 - val_loss: 1.5636 - val_accuracy: 0.3614
Epoch 5/100
45/45 [==============================] - 0s 4ms/step - loss: 1.4672 - accuracy: 0.4142 - val_loss: 1.4723 - val_accuracy: 0.4117
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 1.3736 - accuracy: 0.4767 - val_loss: 1.3867 - val_accuracy: 0.4684
Epoch 7/100
45/45 [==============================] - 0s 4ms/step - loss: 1.2848 - accuracy: 0.5149 - val_loss: 1.3014 - val_accuracy: 0.5251
Epoch 8/100
4

## Part 6 - Plotting the accuracy and the loss for both models

### Displaying the accruacy of the two models on the test set

In [81]:
ann_wHW.evaluate(X_test_wHW, y_test_wHW);

20/20 [==============================] - 0s 2ms/step - loss: 0.2307 - accuracy: 0.9254


In [82]:
ann_nHW.evaluate(X_test_nHW, y_test_nHW);

 1/20 [>.............................] - ETA: 0s - loss: 0.2523 - accuracy: 0.8750

20/20 [==============================] - 0s 2ms/step - loss: 0.1894 - accuracy: 0.9319
